In [ ]:
from lib.ev3 import EV3
from lib.vstream import VideoStream
from lib.logger import Logger


# Define the connection ports of the sensors and motors.
touch_port = EV3.PORT_2
color_port = EV3.PORT_3
lmotor_port = EV3.PORT_B
rmotor_port = EV3.PORT_C

ev3 = EV3()
ev3.motor_config(lmotor_port, EV3.LARGE_MOTOR)
ev3.motor_config(rmotor_port, EV3.LARGE_MOTOR)
ev3.sensor_config(touch_port, EV3.TOUCH_SENSOR)
ev3.sensor_config(color_port, EV3.COLOR_SENSOR)

# Calibrate for light intensity on WHITE
print("Press the touch sensor to measure light intensity on WHITE.\n")
ev3.lcd_draw_string('Calibrate white', 0)
while not ev3.touch_sensor_is_pressed(touch_port):
    pass
white = ev3.color_sensor_get_reflect(color_port)
print("WHITE light intensity: {0}".format(white))
ev3.lcd_draw_string('White = {}'.format(white), 1)

while ev3.touch_sensor_is_pressed(touch_port):
    pass

# Calibrate for light intensity on BLACK
print("Press the touch sensor to measure light intensity on BLACK.\n")
ev3.lcd_draw_string('Calibrate black', 0)
while not ev3.touch_sensor_is_pressed(touch_port):
    pass
black = ev3.color_sensor_get_reflect(color_port)
print("BLACK light intensity: {0}".format(black))
ev3.lcd_draw_string('Black = {}'.format(black), 2)

vs = VideoStream(resolution=(20,15), framerate=10, colormode='binary').start()
lg = Logger(top_dir='ml_linetrace_data')

ev3.lcd_draw_string('Push to start.', 0)
while ev3.touch_sensor_is_pressed(touch_port):
    pass

ev3.lcd_draw_string('Go!', 0)

# Control loop
midpoint = (white - black) / 2 + black
while True:
    # Break this loop when the touch sensor was pressed.
    if ev3.touch_sensor_is_pressed(touch_port):
        break
    # Update steer value by P control
    color = ev3.color_sensor_get_reflect(color_port)
    error = color - midpoint
    steer = 1.6 * error
    # Send control values to EV3
    ev3.motor_steer(lmotor_port, rmotor_port, 10, int(steer))
    # Write a pair of (image, steer)
    lg.write(vs.read(), int(steer))

# Finalize
ev3.motor_steer(lmotor_port, rmotor_port, 0, 0)
lg.close()
vs.stop()
ev3.close()